In [1]:
from openfl.interface.interactive_api.federation import Federation

/opt/miniconda3/envs/openfl/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
# !fx director start --disable-tls -c director/director.yaml # start the director

In [31]:
#! sudo apt install gfortran cmake libopenblas-base libopenblas-dev libatlas-base-dev
# ! ENVOY_NAME=envoy_example_name fx envoy start \
#     -n "$ENVOY_NAME" \
#     --disable-tls \
#     --envoy-config-path envoy_config.yaml \
#     -dh localhost \
#     -dp 50051

In [32]:
federation = Federation(client_id='client_id', director_node_fqdn='localhost', director_port='50051',tls=False, cert_chain=None, api_cert=None, api_private_key=None)

In [33]:
federation.get_shard_registry()

{'ENVOY_NAME': {'shard_info': node_info {
    name: "ENVOY_NAME"
  }
  sample_shape: "2"
  target_shape: "1",
  'is_online': True,
  'is_experiment_running': False,
  'last_updated': '2023-02-07 23:11:23',
  'current_time': '2023-02-07 23:11:38',
  'valid_duration': seconds: 120,
  'experiment_name': 'ExperimentName Mock'},
 'rpi': {'shard_info': node_info {
    name: "rpi"
  }
  sample_shape: "2"
  target_shape: "1",
  'is_online': True,
  'is_experiment_running': False,
  'last_updated': '2023-02-07 23:11:28',
  'current_time': '2023-02-07 23:11:38',
  'valid_duration': seconds: 120,
  'experiment_name': 'ExperimentName Mock'}}

In [34]:
from openfl.interface.interactive_api.experiment import FLExperiment
from openfl.interface.interactive_api.experiment import TaskInterface, DataInterface, ModelInterface

In [35]:
fl_experiment = FLExperiment(federation=federation, experiment_name="pranav's test")

In [36]:
from torch import nn
import torch as t
import torch.nn.functional as F
import torch.optim as optim

class XOR(nn.Module):
    def __init__(self, input_dim = 2, output_dim=1):
        super(XOR, self).__init__()
        self.lin1 = nn.Linear(input_dim, 4)
        self.lin2 = nn.Linear(4, output_dim)

    def forward(self, x):
        x = self.lin1(x)
        x = F.sigmoid(x)
        x = self.lin2(x)
        return x


model = XOR()
loss_func = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.2, momentum=0.9)

In [37]:
from openfl.interface.interactive_api.experiment import ModelInterface
MI = ModelInterface(model, optimizer, framework_plugin='openfl.plugins.frameworks_adapters.pytorch_adapter.FrameworkAdapterPlugin')

In [38]:
from torch.utils.data import DataLoader


class CustomDataInterface(DataInterface):
    def __init__(self, **kwargs):
        # Initialize superclass with kwargs: this array will be passed
        # to get_data_loader methods
        super().__init__(**kwargs)

    @property
    def shard_descriptor(self):
        return self._shard_descriptor

    @shard_descriptor.setter
    def shard_descriptor(self, shard_descriptor):
        """
        Describe per-collaborator procedures or sharding.

        This method will be called during a collaborator initialization.
        Local shard_descriptor  will be set by Envoy.
        """
        self._shard_descriptor = shard_descriptor

        self.train_set = self._shard_descriptor.get_dataset('train')
        self.valid_set = self._shard_descriptor.get_dataset('val')


    def get_train_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        return DataLoader(
            self.train_set, batch_size=4, shuffle=True
        )
    def get_valid_loader(self, **kwargs):
        """
        Output of this method will be provided to tasks with optimizer in contract
        """
        return DataLoader(
            self.valid_set, batch_size=4, shuffle=True
        )

    def get_train_data_size(self):
        """
        Information for aggregation
        """
        return len(self.train_set)

    def get_valid_data_size(self):
        """
        Information for aggregation
        """
        return len(self.valid_set)

DI = CustomDataInterface()

In [39]:
import numpy as np
import tqdm

TI = TaskInterface()

task_settings = {
    'batch_size': 32,
}
@TI.add_kwargs(**task_settings)
@TI.register_fl_task(model='model_', data_loader='train_loader',
                     device='device', optimizer='optim')
def train(model_, train_loader, optim, device, batch_size):
    train_loader = tqdm.tqdm(train_loader, desc="train")
    model_.train()
    model_.to(device)

    losses = []

    for data, target in train_loader:
        optim.zero_grad()
        y_hat = model(data.to(dtype=t.float))
        loss = F.mse_loss(y_hat[:,0], target.to(dtype=t.float))
        loss.backward()
        optim.step()
        losses.append(loss.mean().item())
        # train_loader.set_postfix({'Loss', loss.mean().item()})
    return {'train_loss': np.mean(losses),}

@TI.register_fl_task(model='model_', data_loader='val_loader', device='device')
def validate(model_, val_loader, device):
    device = t.device('cpu')
    model_.eval()
    model_.to(device)

    val_loader = tqdm.tqdm(val_loader, desc="validate")
    val_score = 0
    total_samples = 0

    with t.no_grad():
        for data, target in val_loader:
            samples = target.shape[0]
            total_samples += samples
            # data, target = t.tensor(data).to(device), t.tensor(target).to(device, dtype=t.int64)
            output = model_(data.to(dtype=t.float))
            pred = output.argmax(dim=1)
            val_score += pred.eq(target.to()).sum().cpu().numpy()

    return {'acc': val_score / total_samples,}

In [40]:
fl_experiment.start(
    model_provider=MI,
    data_loader=DI,
    task_keeper=TI,
    rounds_to_train=3,
    opt_treatment='CONTINUE_GLOBAL'
)

[23:11:42] INFO     Building 🡆 Object CloudpickleSerializer from openfl.plugins.interface_serializer.cloudpickle_serializer Module.                  ]8;id=43477;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=118593;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py#171\171]8;;\

           INFO     Building 🡆 Object FrameworkAdapterPlugin from openfl.plugins.frameworks_adapters.pytorch_adapter Module.                         ]8;id=440788;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=219213;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py#171\171]8;;\

           INFO     Starting experiment!                                                                                                       ]8;id=676979;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=300867;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py#245\245]8;;\

           INFO     FL-Plan hash is 44f33172c732566ceb979122b3fc8f7578840734ffc4691e0003d0a99b86977fa8d135d2d5884ee148b5dd882029b152                 ]8;id=191113;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=76045;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py#235\235]8;;\

           INFO     FL-Plan hash is 44f33172c732566ceb979122b3fc8f7578840734ffc4691e0003d0a99b86977fa8d135d2d5884ee148b5dd882029b152                 ]8;id=110387;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=729848;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py#235\235]8;;\

...

           INFO     Building 🡆 Object FrameworkAdapterPlugin from openfl.plugins.frameworks_adapters.pytorch_adapter Module.                         ]8;id=945384;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py\plan.py]8;;\:]8;id=910402;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/federated/plan/plan.py#171\171]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=306292;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=634907;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/utilities/utils.py#172\172]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=791329;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=254859;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/utilities/utils.py#172\172]8;;\

           INFO     SetNewExperiment                                                                                                      ]8;id=134292;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/transport/grpc/director_client.py\director_client.py]8;;\:]8;id=189956;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/transport/grpc/director_client.py#209\209]8;;\

           INFO     Experiment was submitted to the director!                                                                                  ]8;id=920809;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=897428;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py#259\259]8;;\

In [41]:
fl_experiment.stream_metrics()

[23:27:05] METRIC   Round 0, collaborator trained train result train_loss:  0.833483                                                           ]8;id=375108;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=420888;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py#144\144]8;;\

           METRIC   Round 0, collaborator validate_local locally_tuned_model_validate result acc:   0.484000                                   ]8;id=659124;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=810371;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py#144\144]8;;\

           METRIC   Round 0, collaborator validate_agg aggregated_model_validate result acc:        0.484000                                   ]8;id=834066;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py\experiment.py]8;;\:]8;id=210268;file:///opt/miniconda3/envs/openfl/lib/python3.9/site-packages/openfl/interface/interactive_api/experiment.py#144\144]8;;\

_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Socket closed"
	debug_error_string = "{"created":"@1675792756.701190328","description":"Error received from peer ipv6:[::1]:50051","file":"src/core/lib/surface/call.cc","file_line":903,"grpc_message":"Socket closed","grpc_status":14}"
>

In [12]:
fl_experiment.remove_experiment_data()